In [1]:

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d110e2fe011d44d489dffcc5194a59c5 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='eb_teHef1zl2WqhjGL8xoD85lSXUh6_9VurXa8loCIlM',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d110e2fe011d44d489dffcc5194a59c5.get_object(Bucket='sepsisprediction-donotdelete-pr-wodmbegr9uodno',Key='data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

raw_data = pd.read_csv(body)
raw_data.head()


,住院流水号,时间戳,C反应蛋白,白细胞数量,国际标准化比值,血浆降钙素原,血浆总胆红素,血小板,血氨,碱性磷酸酶,...,肌酐,纤维蛋白原,血红蛋白,血乳酸,镁,磷,钾,二氧化碳分压,血沉,诊断结果
0,2017-01159418-0,2,35.109718,14.201427,1.304449,0.914241,25.86681,356.877049,58.478641,142.766513,...,75.59633,3.172638,88.0,0.8,1.053805,0.914241,3.1,35.252899,32.490741,0
1,2017-01159418-0,6,56.700000,8.300000,0.980000,0.914241,8.40000,214.000000,58.478641,92.000000,...,21.00000,6.460000,214.0,0.8,1.053805,0.914241,3.1,35.252899,32.490741,0
2,2017-01159418-0,16,56.700000,8.300000,0.980000,0.914241,8.40000,214.000000,58.478641,92.000000,...,21.00000,6.460000,214.0,0.8,1.053805,0.914241,3.1,35.252899,14.000000,0
3,2017-01159418-0,19,56.700000,8.300000,0.980000,0.914241,8.40000,214.000000,58.478641,92.000000,...,21.00000,6.460000,214.0,0.8,1.053805,0.914241,3.1,35.252899,14.000000,0
4,2017-01159418-0,68,14.200000,5.100000,0.980000,0.914241,8.40000,412.000000,58.478641,92.000000,...,21.00000,6.460000,214.5,0.8,1.053805,0.914241,3.1,35.252899,14.000000,0


In [2]:

body = client_d110e2fe011d44d489dffcc5194a59c5.get_object(Bucket='sepsisprediction-donotdelete-pr-wodmbegr9uodno',Key='timeline.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

time_stamp = pd.read_csv(body)
time_stamp.head()


,住院流水号,时间戳
0,2016-0798830-0,1
1,2016-08107700-0,1
2,2016-08109775-0,1
3,2016-10127091-0,1
4,7717785,1


In [4]:
# Manipulate inputs with incorrect datatype eg. string to numeric
raw_data[u'C反应蛋白'] == '<0.5'
raw_data[raw_data[u'C反应蛋白'] == '<0.50'] = 0.5
raw_data[raw_data[u'C反应蛋白'] == '<0.5'] = 0.5
raw_data[raw_data[u'C反应蛋白'] == '>200.0'] = 200

#raw_data[raw_data[u'天门冬氨酸氨基转氨酶'] == '140 A'] = 140

raw_data[raw_data[u'纤维蛋白原'] == '<0.60'] = 0.6

In [2]:
# Checking inputs datatype
import sys
feature_name = raw_data.axes[1]
for feature in feature_name:
    if feature not in [u'住院流水号', u'时间戳']:
        print(feature)
        try:
            raw_data[feature].astype('float64')
            print("datatype transform successfully")
        except ValueError:
            print(sys.exc_info())


C反应蛋白
datatype transform successfully
白细胞数量
datatype transform successfully
国际标准化比值
datatype transform successfully
血浆降钙素原
datatype transform successfully
血浆总胆红素
datatype transform successfully
血小板
datatype transform successfully
血氨
datatype transform successfully
碱性磷酸酶
datatype transform successfully
天门冬氨酸氨基转氨酶
datatype transform successfully
直接胆红素
datatype transform successfully
钙
datatype transform successfully
肌酐
datatype transform successfully
纤维蛋白原
datatype transform successfully
血红蛋白
datatype transform successfully
血乳酸
datatype transform successfully
镁
datatype transform successfully
磷
datatype transform successfully
钾
datatype transform successfully
二氧化碳分压
datatype transform successfully
血沉
datatype transform successfully
诊断结果
datatype transform successfully


In [6]:
# Remove duplicate timestamp
n_row = time_stamp.shape[0]
new_data = []
feature_name = raw_data.axes[1]
for i in range(n_row):
    line = time_stamp.iloc[i]
    patient_no, timestamp = line[u'住院流水号'], line[u'时间戳']
    lines = raw_data[raw_data[u'住院流水号'] == patient_no]
    lines = lines[lines[u'时间戳'] == timestamp]
    data = [patient_no, timestamp]
    for feature in feature_name:
        if feature not in [u'住院流水号', u'时间戳']:
            values = lines[feature]
            values = values.astype('float64')
            data.append(values.mean())
    new_data.append(data)
new_data = pd.DataFrame(new_data, columns = feature_name)
new_data.head()

,住院流水号,时间戳,C反应蛋白,白细胞数量,国际标准化比值,血浆降钙素原,血浆总胆红素,血小板,血氨,碱性磷酸酶,...,钙,肌酐,纤维蛋白原,血红蛋白,血乳酸,镁,磷,钾,二氧化碳分压,血沉
0,2016-0798830-0,1,NaN,22.5,NaN,NaN,NaN,464.0,NaN,NaN,...,1.050,NaN,NaN,167.000000,0.8,NaN,NaN,3.1,42.003454,NaN
1,2016-08107700-0,1,14.00,18.6,NaN,NaN,NaN,636.0,NaN,NaN,...,NaN,NaN,NaN,198.500000,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-08109775-0,1,0.55,10.1,NaN,NaN,NaN,334.0,NaN,NaN,...,1.100,NaN,NaN,187.333333,3.9,NaN,NaN,4.4,47.000000,NaN
3,2016-10127091-0,1,NaN,11.1,NaN,NaN,NaN,423.0,NaN,NaN,...,NaN,NaN,NaN,225.500000,NaN,NaN,NaN,NaN,NaN,NaN
4,7717785,1,NaN,11.2,1.26,NaN,NaN,238.0,NaN,NaN,...,1.415,NaN,1.52,219.400000,2.6,NaN,NaN,3.6,52.429311,NaN


In [7]:
# Save new_data to the storage
from project_lib import Project
project = Project(project_id="54710414-eeb2-42df-820a-698b93f50f84", project_access_token="p-594d39b7f91d72f663da93ed52c36b329ac8c2c7")
project.save_data('data_numeric.csv', new_data.to_csv(header=True, index=False), overwrite=True)

{'asset_id': '4bcb83b2-ccae-4448-860f-56353dfecf52',
 'bucket_name': 'sepsisprediction-donotdelete-pr-wodmbegr9uodno',
 'file_name': 'data_numeric.csv',
 'message': 'File data_numeric.csv has been written successfully to the associated OS'}